Replace `key` with your openai key (do not use quotations)

In [ ]:
#%env OPENAI_API_KEY=REDACTED

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [3]:
import os
import yaml
from openai import OpenAI
client = OpenAI()


In [4]:
def generate_prompts_sliders(slider_query,
                             file_name_to_save=None,
                             temperature=0.2,
                             max_tokens=256,
                             frequency_penalty=0.0,
                             model="gpt-4-turbo-preview",
                             verbose=False,
                             save=True):
    '''
    A function to automatically build prompts for text sliders using GPT4 (or any other openAI model).

    Inputs
    ------
    slider_query (str): A natural language query describing the slider effects the user desired (eg: "I want to make people older")
    file_name_to_save (str) (optional): a full name of the yaml file a user desires. If left as None, a name will be chosen by GPT
    temperature (float) (optional): GPT temperature parameter (use smaller values for less randomness)
    max_tokens (int) (optional): GPT output token limit
    frequency_penalty (float) (optional): GPT frequency penalty
    model (str) (optional): The model class from openAI. By default uses GPT-4-Turbo
    verbose (bool) (optional): A flag to print intermediate responses by GPT
    save (bool) (optional): A flag to save the prompts to a destination path
    '''
    gpt_assistant_prompt =  '''You are an expert in prompting text-image generation models. Given a concept to edit, your task is to generate 4 detailed prompts.
                            1. Target prompt: a prompt that describes the target class which the concept edit is intended to modify (for example, to edit the concept "professional" the target concept is "person". Leave it empty if the target concept is too large. For example if user asks for their generations to be more futuristic, since all the images have to be edited, just leave the target ""
                            2. Positive prompt: a detailed prompt that describes the extreme positive end of the edit concept with the target concept included (for example, "person, very professional, blazer, neat, organized)"
                            3. Negative prompt: a detailed prompt that describes the extreme negative end of the edit concept with the target concept included (for example, "person, non-professional, ragidy, unkempt"). This is optional, you can leave it "" if there is no obvious negative prompt.
                            4. Preservation prompt: a prompt (must be comma separated) that describes any concepts except the ones to edit that should be preserved when making the edit without the target concept included (for example, "white, black, indian, asian, hispanic; male, female" as the race or gender of a person may be changed when we edit the professionalism.). This should not include edit concepts and should not include any of the positive or negative concepts. if there are no obvious entanglement issues with the edit, leave the prompt ""
                            make preservation prompt comma seperated for each class of perservation. For example if you want to preserve both race and gender, then give something like "white race, black race, indian race, asian race; male, female"

                            All the prompts must be strictly string type. Be specific. Do not use any alphanumeric symbols.

                            This is an example template for your response when asked to generate prompts for making people smile:
                            Target: person
                            Positive: person, smiling, happy face, big smile
                            Negative: person, frowning, grumpy, sad
                            Preservation: white, black, indian, asian, hispanic ; male, female
                            Name: person_age_GPT

                            Here is another example template for your response when asked - "I want to make images more detailed":
                            Target:
                            Positive: highly detailed, intricate patterns, fine textures, realistic shading
                            Negative: simplistic, minimalistic, abstract, rough outlines
                            Preservation:
                            Name: detailed_GPT
                            '''
    gpt_user_prompt = slider_query
    gpt_prompt = gpt_assistant_prompt, gpt_user_prompt
    message=[{"role": "assistant", "content": gpt_assistant_prompt}, {"role": "user", "content": gpt_user_prompt}]

    response = client.chat.completions.create(
        model= model,
        messages = message,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty
    )
    content = response.choices[0].message.content
    if verbose:
        print(content)
    prompts = content.splitlines()
    result = {}
    result['target'] = ""
    result['positive'] = ""
    result['unconditional'] = ""
    result['neutral'] = ""
    for prompt in prompts:
        key = prompt.split(':')
        if key[0].lower().strip() == 'preservation':
            final_attributes = []
            attributes = key[1].split(';')
            for attribute in attributes:
                if len(attribute.strip()) == 0:
                    continue
                final_attributes.append(attribute.strip().split(','))
        elif key[0].lower().strip() == 'name':
            name = key[1].strip()
    for prompt in prompts:
        key = prompt.split(':')
        if len(key)!=2:
            continue
        if key[0].lower().strip() == 'target':
            result['target'] = key[1].strip()
        elif key[0].lower().strip() == 'positive':
            result['positive'] = key[1].strip()
        elif key[0].lower().strip() == 'negative':
            result['unconditional'] = key[1].strip()
    result['neutral'] = result['target']
    results = [result]

    for attribute_class in final_attributes:
        results_final  = []
        for attribute in attribute_class:
            for result in results:
                r = {}
                for key in result.keys():
                    r[key] = attribute.strip() + f' {result[key].strip()}'
                    r[key] = r[key].strip()
                results_final.append(r)

        results = results_final
    results_final = []
    for result in results:
        r_final = result
        r_final['guidance'] = 2
        r_final['rank'] = 2
        r_final['action'] = 'enhance'
        r_final['resolution'] = 512
        r_final['dynamic_resolution'] = False
        r_final['batch_size'] = 1
        results_final.append(r_final)
    if file_name_to_save is None:
        if name is None:
            file_name_to_save = 'custom-prompts-GPT.yaml'
        else:
            file_name_to_save = f'prompts-{name}.yaml'
    if save:
        with open(f'{file_name_to_save}', 'w+') as f:
            yaml.dump(results_final, f, allow_unicode=True, sort_keys=False)
        if verbose:
            print(f'Prompt file saved to: "trainscripts/textsliders/data/{file_name_to_save}"')
        return f'trainscripts/textsliders/data/{file_name_to_save}'

In [5]:
query = "I want to build a slider to make people chubbier"
generate_prompts_sliders(slider_query=query, model="gpt-4-turbo-preview", save=True, verbose=True)

Target: person
Positive: person, chubby, fuller figure, round face
Negative: person, slim, thin, angular features
Preservation: white, black, indian, asian, hispanic ; male, female
Name: person_chubbiness_GPT
Prompt file saved to: "trainscripts/textsliders/data/prompts-person_chubbiness_GPT.yaml"


'trainscripts/textsliders/data/prompts-person_chubbiness_GPT.yaml'